In [1]:
import requests
import os
import time
import pandas as pd
import aiohttp
import asyncio
import json
import nest_asyncio
nest_asyncio.apply()

In [2]:
TARGET = '../Thung-Song-final/1final/'
OUTPUT = '../Thung-Song-final-SunriseSunset/1duration'

In [3]:
if not os.path.exists(OUTPUT):
    os.makedirs(OUTPUT)

In [4]:
# Collect all Excel files in the target directory
excel_files = [f for f in os.listdir(TARGET) if f.endswith('.xlsx')]

In [5]:
# Initialize an empty list to store dataframes
dataframes = []

In [6]:
if os.path.isdir(TARGET):
    for filename in os.listdir(TARGET):
        file_path = os.path.join(TARGET, filename)
        if filename.endswith('.xlsx'):
            print(filename)
            df = pd.read_excel(file_path)
            # print(df.head())
            dataframes.append(df)
else:
    print(f'{TARGET} is not a directory.')

2500-Pak Phraek.xlsx
2501-Chamai.xlsx
2502-Nong Hong.xlsx
2503-Na Pho.xlsx
2504-Khuan Krot.xlsx
2505-Na Mai Phai.xlsx
2506-Thi Wang.xlsx
2507-Kapang.xlsx
2508-Khao Ro.xlsx
2509-Na Luang Sen.xlsx
2510-Namtok.xlsx
2511-Khao Khao.xlsx
2512-Tham Yai.xlsx


In [7]:
# Combine all dataframes into one
df = pd.concat(dataframes, ignore_index=True)

In [8]:
temp = []
responses = []

In [9]:
for i, data in df.iterrows():
    lat = data['latitude']
    lon = data['longitude']
    date = str(data['date']).replace('/', '-')
    temp.append([lat, lon, date])

In [10]:
async def fetch_data(session, url):
    async with session.get(url) as response:
        return await response.json()

async def main():
    urls = [f"https://api.sunrisesunset.io/json?lat={lat}&lng={lon}&date_start={date}&date_end={date}" for lat, lon, date in temp]
    global responses

    async with aiohttp.ClientSession() as session:
        tasks = [fetch_data(session, url) for url in urls]
        responses = await asyncio.gather(*tasks)
        

In [11]:
asyncio.run(main())

KeyboardInterrupt: 

In [ ]:
responses = [d['results'][0] for d in responses]
df_api = pd.DataFrame.from_dict(responses)
df_api = df_api[['date', 'sunrise', 'sunset']]
df_api['date'] = df_api['date'].replace('-', '/', regex=True)
df_api

In [ ]:
df['sunrise'] = pd.to_datetime(df_api['sunrise'])
df['sunset'] = pd.to_datetime(df_api['sunset'])
df['durationOfDay'] = (df['sunset'] - df['sunrise']).dt.total_seconds()/3600
df.head()

In [ ]:
df['sunrise'] = df['sunrise'].dt.time
df['sunset'] = df['sunset'].dt.time
df

In [ ]:
df = df[['date', 'latitude', 'longitude', 'uvb', 'e', 'stl1', 'sp', 'tp', 'd2m', 'minTemp', 'meanTemp', 'maxTemp', 'RH', 'WS', 'sunrise', 'sunset', 'durationOfDay']]
df.head()

In [ ]:
output_file = os.path.join(OUTPUT, 'combined_sunrise_sunset_duration.xlsx')
df.to_excel(output_file, index=False)

### for loop

In [ ]:
import os
import pandas as pd
import aiohttp
import asyncio
import nest_asyncio

nest_asyncio.apply()

TARGET = '../Thung-Song-final/1final/'
OUTPUT = '../Thung-Song-final-SunriseSunset/1duration'

# Ensure the output directory exists
if not os.path.exists(OUTPUT):
    os.makedirs(OUTPUT)

# Collect all Excel files in the target directory
excel_files = [f for f in os.listdir(TARGET) if f.endswith('.xlsx')]

# Initialize an empty list to store dataframes
dataframes = []

# Load and print each Excel file, appending it to the dataframes list
if os.path.isdir(TARGET):
    for filename in excel_files:
        file_path = os.path.join(TARGET, filename)
        print(f'Loading {filename}')
        df = pd.read_excel(file_path)
        dataframes.append(df)
else:
    print(f'{TARGET} is not a directory.')

# Combine all dataframes into one
df = pd.concat(dataframes, ignore_index=True)

# Display the combined dataframe
print(df.head())

# Extract latitude, longitude, and date data for API requests
temp = []
responses = []

for i, data in df.iterrows():
    lat = data['latitude']
    lon = data['longitude']
    date = str(data['date']).replace('/', '-')
    temp.append([lat, lon, date])

# Asynchronous function to fetch data from the API
async def fetch_data(session, url):
    async with session.get(url) as response:
        return await response.json()

# Asynchronous function to manage API requests
async def main():
    urls = [f"https://api.sunrisesunset.io/json?lat={lat}&lng={lon}&date_start={date}&date_end={date}" for lat, lon, date in temp]
    global responses

    async with aiohttp.ClientSession() as session:
        tasks = [fetch_data(session, url) for url in urls]
        responses = await asyncio.gather(*tasks)

# Run the asynchronous function
asyncio.run(main())

# Process the API responses and convert them to a dataframe
responses = [d['results'][0] for d in responses]
df_api = pd.DataFrame.from_dict(responses)
df_api = df_api[['date', 'sunrise', 'sunset']]
df_api['date'] = df_api['date'].replace('-', '/', regex=True)

# Merge the API data with the original dataframe
df['sunrise'] = pd.to_datetime(df_api['sunrise'])
df['sunset'] = pd.to_datetime(df_api['sunset'])
df['durationOfDay'] = (df['sunset'] - df['sunrise']).dt.total_seconds() / 3600

# Convert sunrise and sunset columns to time format
df['sunrise'] = df['sunrise'].dt.time
df['sunset'] = df['sunset'].dt.time

# Select and reorder the relevant columns
df = df[['date', 'latitude', 'longitude', 'uvb', 'e', 'stl1', 'sp', 'tp', 'd2m', 'minTemp', 'meanTemp', 'maxTemp', 'RH', 'WS', 'sunrise', 'sunset', 'durationOfDay']]

# Save the updated dataframe to a new Excel file
output_file = os.path.join(OUTPUT, 'combined_sunrise_sunset_duration.xlsx')
df.to_excel(output_file, index=False)

print(f'Saved updated dataframe to {output_file}')


In [ ]:
import os
import pandas as pd
import aiohttp
import asyncio
import nest_asyncio
from aiohttp import ClientSession
from aiohttp.client_exceptions import ClientConnectorError, ClientOSError, ServerDisconnectedError
import time

nest_asyncio.apply()

TARGET = '../Thung-Song-final/3final/'
OUTPUT = '../Thung-Song-final-SunriseSunset/3duration'

# Ensure the output directory exists
if not os.path.exists(OUTPUT):
    os.makedirs(OUTPUT)

# Collect all Excel files in the target directory
excel_files = [f for f in os.listdir(TARGET) if f.endswith('.xlsx')]

# Function to process each Excel file
async def process_file(file_path, filename):
    print(f'Loading {filename}')
    df = pd.read_excel(file_path)

    # Extract latitude, longitude, and date data for API requests
    temp = []
    for i, data in df.iterrows():
        lat = data['latitude']
        lon = data['longitude']
        date = str(data['date']).replace('/', '-')
        temp.append([lat, lon, date])

    # Asynchronous function to fetch data from the API with retry logic
    async def fetch_data(session, url, retries=3, backoff_factor=1.0):
        for attempt in range(retries):
            try:
                async with session.get(url) as response:
                    if response.status == 200:
                        return await response.json()
                    else:
                        print(f"Error response {response.status} for URL: {url}")
            except (ClientConnectorError, ClientOSError, ServerDisconnectedError) as e:
                print(f"Request failed: {e}. Retrying ({attempt + 1}/{retries})...")
                await asyncio.sleep(backoff_factor * (2 ** attempt))
        print(f"Failed to fetch data from {url} after {retries} retries.")
        return None

    # Asynchronous function to manage API requests
    async def fetch_all_data():
        urls = [f"https://api.sunrisesunset.io/json?lat={lat}&lng={lon}&date_start={date}&date_end={date}" for lat, lon, date in temp]
        responses = []
        async with aiohttp.ClientSession() as session:
            tasks = [fetch_data(session, url) for url in urls]
            responses = await asyncio.gather(*tasks)
        return [response for response in responses if response is not None]

    responses = await fetch_all_data()

    if not responses:
        print(f"No data fetched for {filename}. Skipping file.")
        return

    # Process the API responses and convert them to a dataframe
    responses = [d['results'][0] for d in responses]
    df_api = pd.DataFrame.from_dict(responses)
    df_api = df_api[['date', 'sunrise', 'sunset']]
    df_api['date'] = df_api['date'].replace('-', '/', regex=True)

    # Merge the API data with the original dataframe
    df['sunrise'] = pd.to_datetime(df_api['sunrise'])
    df['sunset'] = pd.to_datetime(df_api['sunset'])
    df['durationOfDay'] = (df['sunset'] - df['sunrise']).dt.total_seconds() / 3600

    # Convert sunrise and sunset columns to time format
    df['sunrise'] = df['sunrise'].dt.time
    df['sunset'] = df['sunset'].dt.time

    # Select and reorder the relevant columns
    df = df[['date', 'latitude', 'longitude', 'uvb', 'e', 'stl1', 'sp', 'tp', 'd2m', 'minTemp', 'meanTemp', 'maxTemp', 'RH', 'WS', 'sunrise', 'sunset', 'durationOfDay']]

    # Save the updated dataframe to a new Excel file
    output_file = os.path.join(OUTPUT, f'{filename.split(".")[0]}-durationOfDay.xlsx')
    df.to_excel(output_file, index=False)

    print(f'Saved updated dataframe to {output_file}')

# Run the processing for each file
async def main():
    tasks = [process_file(os.path.join(TARGET, filename), filename) for filename in excel_files]
    await asyncio.gather(*tasks)

asyncio.run(main())


In [ ]:
import os
import pandas as pd
import aiohttp
import asyncio
import nest_asyncio
from aiohttp import ClientSession
from aiohttp.client_exceptions import ClientConnectorError, ClientOSError, ServerDisconnectedError
import time
nest_asyncio.apply()

In [2]:
for j in range(1,4):
    TARGET = f'../Chanae-final/{j}final/'
    OUTPUT = f'../Chanae-final-SunriseSunset/{j}duration'

    # Ensure the output directory exists
    if not os.path.exists(OUTPUT):
        os.makedirs(OUTPUT)

    # Collect all Excel files in the target directory
    excel_files = [f for f in os.listdir(TARGET) if f.endswith('.xlsx')]

    # Function to process each Excel file
    async def process_file(file_path, filename):
        print(f'Loading {filename}')
        df = pd.read_excel(file_path)

        # Extract latitude, longitude, and date data for API requests
        temp = []
        for i, data in df.iterrows():
            lat = data['latitude']
            lon = data['longitude']
            date = str(data['date']).replace('/', '-')
            temp.append([lat, lon, date])

        # Asynchronous function to fetch data from the API with retry logic
        async def fetch_data(session, url, retries=3, backoff_factor=1.0):
            for attempt in range(retries):
                try:
                    async with session.get(url) as response:
                        if response.status == 200:
                            return await response.json()
                        else:
                            print(f"Error response {response.status} for URL: {url}")
                except (ClientConnectorError, ClientOSError, ServerDisconnectedError) as e:
                    print(f"Request failed: {e}. Retrying ({attempt + 1}/{retries})...")
                    await asyncio.sleep(backoff_factor * (2 ** attempt))
            print(f"Failed to fetch data from {url} after {retries} retries.")
            return None

        # Asynchronous function to manage API requests
        async def fetch_all_data():
            urls = [f"https://api.sunrisesunset.io/json?lat={lat}&lng={lon}&date_start={date}&date_end={date}" for lat, lon, date in temp]
            responses = []
            async with aiohttp.ClientSession() as session:
                tasks = [fetch_data(session, url) for url in urls]
                responses = await asyncio.gather(*tasks)
            return [response for response in responses if response is not None]

        responses = await fetch_all_data()

        if not responses:
            print(f"No data fetched for {filename}. Skipping file.")
            return

        # Process the API responses and convert them to a dataframe
        responses = [d['results'][0] for d in responses]
        df_api = pd.DataFrame.from_dict(responses)
        df_api = df_api[['date', 'sunrise', 'sunset']]
        df_api['date'] = df_api['date'].replace('-', '/', regex=True)

        # Merge the API data with the original dataframe
        df['sunrise'] = pd.to_datetime(df_api['sunrise'])
        df['sunset'] = pd.to_datetime(df_api['sunset'])
        df['durationOfDay'] = (df['sunset'] - df['sunrise']).dt.total_seconds() / 3600

        # Convert sunrise and sunset columns to time format
        df['sunrise'] = df['sunrise'].dt.time
        df['sunset'] = df['sunset'].dt.time

        # Select and reorder the relevant columns
        df = df[['date', 'latitude', 'longitude', 'uvb', 'e', 'stl1', 'sp', 'tp', 'd2m', 'minTemp', 'meanTemp', 'maxTemp', 'RH', 'WS', 'sunrise', 'sunset', 'durationOfDay']]

        # Save the updated dataframe to a new Excel file
        output_file = os.path.join(OUTPUT, f'{filename.split(".")[0]}-durationOfDay.xlsx')
        df.to_excel(output_file, index=False)

        print(f'Saved updated dataframe to {output_file}')

    # Run the processing for each file
    async def main():
        tasks = [process_file(os.path.join(TARGET, filename), filename) for filename in excel_files]
        await asyncio.gather(*tasks)

    asyncio.run(main())

Loading Chanae.xlsx
Loading Chang-Phueak.xlsx
Loading Du-Song-Yo.xlsx
Loading Phadung-Mat.xlsx


C:\Users\User\AppData\Local\Temp\ipykernel_12096\195814389.py:62: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['sunrise'] = pd.to_datetime(df_api['sunrise'])
C:\Users\User\AppData\Local\Temp\ipykernel_12096\195814389.py:63: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['sunset'] = pd.to_datetime(df_api['sunset'])


Saved updated dataframe to ../Chanae-final-SunriseSunset/1duration\Chang-Phueak-durationOfDay.xlsx


C:\Users\User\AppData\Local\Temp\ipykernel_12096\195814389.py:62: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['sunrise'] = pd.to_datetime(df_api['sunrise'])
C:\Users\User\AppData\Local\Temp\ipykernel_12096\195814389.py:63: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['sunset'] = pd.to_datetime(df_api['sunset'])


Saved updated dataframe to ../Chanae-final-SunriseSunset/1duration\Du-Song-Yo-durationOfDay.xlsx
Saved updated dataframe to ../Chanae-final-SunriseSunset/1duration\Chanae-durationOfDay.xlsx


...

Saved updated dataframe to ../Chanae-final-SunriseSunset/2duration\Chanae-durationOfDay.xlsx
Loading Chanae.xlsx
Loading Chang-Phueak.xlsx
Loading Du-Song-Yo.xlsx
Loading Phadung-Mat.xlsx


C:\Users\User\AppData\Local\Temp\ipykernel_12096\195814389.py:62: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['sunrise'] = pd.to_datetime(df_api['sunrise'])
C:\Users\User\AppData\Local\Temp\ipykernel_12096\195814389.py:63: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['sunset'] = pd.to_datetime(df_api['sunset'])
C:\Users\User\AppData\Local\Temp\ipykernel_12096\195814389.py:62: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['sunrise'] = pd.to_datetime(df_api['sunrise'])
C:\Users\User\AppData\Local\Temp\ipykernel_12096\195814389.py:63: UserWarning: Could not infer f

Saved updated dataframe to ../Chanae-final-SunriseSunset/3duration\Chanae-durationOfDay.xlsx
Saved updated dataframe to ../Chanae-final-SunriseSunset/3duration\Phadung-Mat-durationOfDay.xlsx


C:\Users\User\AppData\Local\Temp\ipykernel_12096\195814389.py:62: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['sunrise'] = pd.to_datetime(df_api['sunrise'])
C:\Users\User\AppData\Local\Temp\ipykernel_12096\195814389.py:63: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['sunset'] = pd.to_datetime(df_api['sunset'])
C:\Users\User\AppData\Local\Temp\ipykernel_12096\195814389.py:62: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['sunrise'] = pd.to_datetime(df_api['sunrise'])
C:\Users\User\AppData\Local\Temp\ipykernel_12096\195814389.py:63: UserWarning: Could not infer f

Saved updated dataframe to ../Chanae-final-SunriseSunset/3duration\Du-Song-Yo-durationOfDay.xlsx
Saved updated dataframe to ../Chanae-final-SunriseSunset/3duration\Chang-Phueak-durationOfDay.xlsx
